# Processing Census 2011 Web Data

## TO DO LIST
- Check all names from the pdfs and dict_cols
- Make a script that combines everything into a one script
- 

In [1]:
import os
import re
import PyPDF2
import pandas as pd
from config import Config
os.getcwd()

'/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/fe/bbs'

In [2]:
census2011 = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/bbs/webtestapplication/userfiles/image/Census2011'


In [3]:
output = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/bbs/'
#Barguna = '../../data/bbs/webtestapplication/userfiles/image/Census2011/Barisal/Barguna/'
#baruna_files = sorted(glob.glob(Barguna + '/*[0-9].pdf'))
#baruna_files

In [6]:
def set_wd(path):
    os.chdir(path)


def check_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False


def get_pdf_files():
    output = {'file': [], 'program': []}
    for path, subdirs, files in os.walk('.'):
        for name in files:
            if bool(re.search(r'_C[0-9][0-9].pdf', name)):
                output['file'].append(os.path.join(path, name))
                output['program'].append(name[-7:-4])
    return pd.DataFrame.from_dict(output)

def parse_files(file, ncol_before, ncol_after, col_names):
    df_file_obj = open(file, 'rb')
    pdf_reader = PyPDF2.PdfFileReader(df_file_obj, strict=False)
    num_pages = pdf_reader.getNumPages()
    pages = ''
    subset = {}
    for page in range(num_pages):
        page_obj = pdf_reader.getPage(page)
        pages = pages + page_obj.extractText()
    pages = pages.split("\n")
    pages = [element.replace(",", "") for element in pages]
    for key, element in enumerate(pages):
        if element.isspace() or not element.strip():
            pages[key] = float('nan')
        if key > 6:
            try:
                digits_check = pages[key - 4:key + 5]
                string_check = digits_check.pop(4)
                digits_check = [int(digit) for digit in digits_check]
                geoname_check = not string_check.isdigit()
                values_check_int = all([str(digit).isdigit() for digit in digits_check])
                geoname_values_check = geoname_check and values_check_int
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    subset[key] = pages[key - ncol_before:key + ncol_after]
            except ValueError:
                digits_check = pages[key - 4:key + 5]
                string_check = digits_check.pop(4)
                geoname_check = not check_float(string_check)
                values_check_float = all([check_float(digit) for digit in digits_check])
                geoname_values_check = geoname_check and values_check_float
                patterns = ['Zila', 'Upazila', 'Ward', 'Union', '*']
                pattern_check = any([pattern in element for pattern in patterns])
                if geoname_values_check or pattern_check:
                    subset[key] = pages[key - ncol_before:key + ncol_after]
            else:
                pass
    subset = pd.DataFrame.from_dict(subset, orient='index', columns=col_names)
    df_file_obj.close()
    return subset

def write_data(input_files, output_dir):
    for subset in input_files['program'].unique():
        tmp = input_files[input_files['program'] == subset]
        out_list = []
        for key, values in enumerate(tmp.values):
            data = parse_files(values[0],
                           ncol_before=Config.parse_parameters[values[1]]['ncol_before'],
                           ncol_after=Config.parse_parameters[values[1]]['ncol_after'],
                           col_names=Config.cols_dict[values[1]])
            out_list.append(data)
        program_df = pd.concat(out_list)
        program_df.to_csv(os.path.join(output_dir,subset+".csv"), index=False, index_label=None)

def main():
    set_wd(path=census2011)
    out = get_pdf_files()
    write_data(input_files=out,output_dir=output)
    return out

In [ ]:
main()

In [ ]:
t1 = parse_files(file=baruna_files[0], 
                     ncol_before=6, 
                     ncol_after=5, 
                     col_names = cols_dict['C01'])
print(t1['ZL'].value_counts())
t1.head()


In [ ]:
t2 = parse_files(file=baruna_files[1], ncol_before=6, ncol_after=9, col_names = cols_dict['C02'])
print(t2['ZL'].value_counts())
t2.head()

In [ ]:
t3 = parse_files(file=baruna_files[2], ncol_before=6, ncol_after=11, col_names = cols_dict['C03'])
print(t3['ZL'].value_counts())
t3.head()

In [ ]:
t4 = parse_files(file=baruna_files[3], ncol_before=6, ncol_after=12, col_names = cols_dict['C04'])
print(t4['ZL'].value_counts())
t4.head()

In [ ]:
t5 = parse_files(file=baruna_files[4], ncol_before=6, ncol_after=11, col_names = cols_dict['C05'])
print(t5['ZL'].value_counts())
t5.head()

In [ ]:
print(cols_dict['C06'])
print(len(cols_dict['C06']))
t6 = parse_files(file=baruna_files[5], ncol_before=6, ncol_after=10, col_names = cols_dict['C06'])
print(t6['ZL'].value_counts())
t6.head()

In [ ]:
print(cols_dict['C07'])
print(len(cols_dict['C07']))
t7 = parse_files(file=baruna_files[6], ncol_before=6, ncol_after=13, col_names = cols_dict['C07'])
print(t7['ZL'].value_counts())
t7.head()

In [ ]:
print(cols_dict['C08'])
print(len(cols_dict['C08']))
t8 = parse_files(file=baruna_files[7], ncol_before=6, ncol_after=13, col_names = cols_dict['C08'])
print(t8['ZL'].value_counts())
t8.head()

In [ ]:
print(cols_dict['C09'])
print(len(cols_dict['C09']))
t9 = parse_files(file=baruna_files[8], ncol_before=6, ncol_after=9, col_names = cols_dict['C09'])
print(t9['ZL'].value_counts())
t9.head()

In [ ]:
print(cols_dict['C10'])
print(len(cols_dict['C10']))
t10 = parse_files(file=baruna_files[9], ncol_before=6, ncol_after=9, col_names=cols_dict['C10'])
print(t10['ZL'].value_counts())
t10.head()

In [ ]:
print(cols_dict['C11'])
print(len(cols_dict['C11']))
t11 = parse_files(file=baruna_files[10], ncol_before=6, ncol_after=10, col_names=cols_dict['C11'])
print(t11['ZL'].value_counts())
t11.head()

In [ ]:
print(cols_dict['C12'])
print(len(cols_dict['C12']))
t12 = parse_files(file=baruna_files[11], ncol_before=6, ncol_after=9, col_names=cols_dict['C12'])
print(t12['ZL'].value_counts())
t11.head()

In [ ]:
print(cols_dict['C12_e'])
print(len(cols_dict['C12_e']))
t12_e = parse_files(file=baruna_files[12], ncol_before=6, ncol_after=9, col_names=cols_dict['C12_e'])
print(t12_e['ZL'].value_counts())
t11.head()

In [ ]:
print(cols_dict['C13'])
print(len(cols_dict['C13']))
t13 = parse_files(file=baruna_files[13], ncol_before=6, ncol_after=7, col_names=cols_dict['C13'])
print(t13['ZL'].value_counts())
t13.head()

In [ ]:
print(cols_dict['C14'])
print(len(cols_dict['C14']))
t14 = parse_files(file=baruna_files[14], ncol_before=6, ncol_after=10, col_names=cols_dict['C14'])
print(t14['ZL'].value_counts())
t14.head()

In [ ]:
print(cols_dict['C15'])
print(len(cols_dict['C15']))
t15 = parse_files(file=baruna_files[15], ncol_before=6, ncol_after=9, col_names=cols_dict['C15'])
print(t15['ZL'].value_counts())
t15.head()